In [1]:
import os
import re
from datetime import datetime
from ipynb.fs.full.useful_functions_for_plotting_and_reading import define_parameters_from_filename
import os
import numpy as np

### Funzione per SOSTITUIRE una certa sottostringa in tutti i file di una directory.

In [2]:
def rename_files_in_directory(directory_path, old_substring, new_substring):
    """
    Sostituisce una sottostringa nei nomi di tutti i file di una directory,
    ma solo per i file con estensione .csv o .txt..
    
    Args:
        directory_path (str): Percorso alla directory.
        old_substring (str): Sottostringa da sostituire.
        new_substring (str): Nuova sottostringa per la sostituzione.
    """
    # Verifica che la directory esista
    if not os.path.isdir(directory_path):
        print(f"La directory '{directory_path}' non esiste.")
        return

    # Itera sui file nella directory
    for filename in os.listdir(directory_path):
        # Controlla se la sottostringa è presente nel nome del file
        if old_substring in filename and (filename.endswith('.csv') or filename.endswith('.txt')):
            # Crea il nuovo nome del file
            new_filename = filename.replace(old_substring, new_substring)
            # Percorsi completi per rinominare
            old_filepath = os.path.join(directory_path, filename)
            new_filepath = os.path.join(directory_path, new_filename)
            # Rinomina il file
            os.rename(old_filepath, new_filepath)
            print(f"Rinominato: {filename} -> {new_filename}")

#### Esempio di utilizzo

In [3]:
directory = "./"
old_substring = "['000001', '000010', '000100', '001000', '010000', '100000']" # Sottostringa da sostituire
new_substring = "all1" # Nuova sottostringa

rename_files_in_directory(directory, old_substring, new_substring)

Rinominato: k_L=n@21-12_dim6_mail1_['000001', '000010', '000100', '001000', '010000', '100000']_random_p3_20ra_k0.334_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44].csv -> k_L=n@21-12_dim6_mail1_all1_random_p3_20ra_k0.334_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44].csv
Rinominato: k_L=n@21-12_dim6_mail1_['000001', '000010', '000100', '001000', '010000', '100000']_random_p3_20ra_k0.334_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44]_data.txt -> k_L=n@21-12_dim6_mail1_all1_random_p3_20ra_k0.334_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44]_data.txt
Rinominato: k_L=n@21-12_dim6_mail2_['000001', '000010', '000100', '001000', '010000', '100000']_random_p3_20ra_k0.5_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44].csv -> k_L=n@21-12_dim6_mail2_all1_random_p3_20ra_k0.5_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44].csv
Rinominato: k_L=n@21-12_dim6_mail2_['000001', '000010', '000100', '001000', '010000', '100000']_random_p3_20ra_k0.5_BOUNDS[0,2pi]x[-44,44]_pars0[0,2pi]x[-44,44]_data.txt -> k_L=n@21-12_di

### Funzione per ELIMINARE PRIMA DI una certa sottostringa in tutti i file di una directory.¶

In [10]:
def remove_before_substring_in_directory(directory_path, substring):
    """
    Rimuove tutto ciò che si trova prima di una determinata sottostringa, mantenendo la sottostringa stessa e tutto il resto
    nel nome di tutti i file di una directory, ma solo per i file con estensione .csv o .txt.
    
    Args:
        directory_path (str): Percorso alla directory.
        substring (str): Sottostringa dopo cui mantenere il nome del file, inclusa.
    """
    # Verifica che la directory esista
    if not os.path.isdir(directory_path):
        print(f"La directory '{directory_path}' non esiste.")
        return

    # Itera sui file nella directory
    for filename in os.listdir(directory_path):
        # Controlla se la sottostringa è presente nel nome del file e se è un file .csv o .txt
        if substring in filename and (filename.endswith('.csv') or filename.endswith('.txt')):
            # Trova la posizione della sottostringa e rimuove tutto prima di essa (incluso)
            start_index = filename.find(substring)
            new_filename = filename[start_index:]  # Mantieni la sottostringa e tutto dopo di essa
            # Percorsi completi per rinominare
            old_filepath = os.path.join(directory_path, filename)
            new_filepath = os.path.join(directory_path, new_filename)
            # Rinomina il file
            os.rename(old_filepath, new_filepath)
            print(f"Rinominato: {filename} -> {new_filename}")


In [11]:
# # Choose the directory and remove substrings before a certain substring
# directory = "./FILES_PER-IMMAGINI-LATEX/tutte-le-istanze_all1_50ra/"
# remove_before_substring_in_directory(directory, "serietutteistanze50")

### Funzione per leggere i file txt, scorrerli per trovare il migliore tentativo e i vari dati associati.

In [12]:
def analyze_attempts(datafile_path, define_parameters=True):
    """
    Analizza un file per identificare i parametri associati al miglior tentativo.
    
    Args:
        file_path (str): Percorso al file di input.
    
    Returns:
        dict: Dizionario contenente i dati estratti.
    """
    file_path = datafile_path.split("_data")[0] + ".csv"

    n, instance, init_name, p, ra, k = define_parameters_from_filename(file_path, verbose=True)
    
    with open(datafile_path, "r") as file:
        lines = file.readlines()

    # Iterare attraverso i tentativi per trovare il migliore dal fondo
    for line in reversed(lines):
        if "Attempt that reached the best result" in line:
            best_attempt = int(re.search(r"#(\d+)", line).group(1))

    # Trovare i parametri del migliore 
    best_idx = len(lines)
    for idx,line in enumerate(lines):
        if idx == 0: 
            datetime_str = line.split("= ")[1]
        if idx == 1:
            p = int(line.split("=")[1])
        if idx == 3:
            init_name = line.split(":")
        if f"----- {best_attempt}/{ra} random_attempts -----" in line:
            best_idx = idx
        if idx == best_idx+2:
            best_E_min = float(line.split('E_min = ')[1])
        if idx == best_idx+3:
            best_betas = list(map(float, re.search(r"betas = \[(.*?)\]", line).group(1).split(", ")))
            best_gammas = list(map(float, re.search(r"gammas = \[(.*?)\]",  line).group(1).split(", ")))
            break

    # Estrarre il tempo totale
    elapsed_time = float(re.search(r"TOTAL ELAPSED TIME: ([\d.]+)", "".join(lines)).group(1))

    # Restituire i risultati in un dizionario
    return {
        "current_datetime": datetime_str,
        "p": p,
        "init_name": init_name,
        "best_attempt": best_attempt,
        "best_E_min": best_E_min,
        "betas": best_betas,
        "gammas": best_gammas,
        "total_elapsed_time_minutes": elapsed_time,
    }

In [13]:
# # Esempio di utilizzo della funzione
# datafile_path = "./solotol1e4_@09-12_dim6_mail3_all1_random_p3_50ra_k1_BOUNDS[0,2pi]x[-20,20]_pars0[0,2pi]x[-20,20]_data.txt"
# results = analyze_attempts(datafile_path)
# for key, value in results.items():
#     print(f"{key}: {value}")

### Applico la funzione precedente a tutti i file della directory

In [14]:
def analyze_all_files_with_substring(directory_path, substrings):
    """
    Applica la funzione analyze_attempts a tutti i file nella cartella corrente
    che contengono una determinata sottostringa.
    
    Args:
        directory_path (str): Percorso della cartella da analizzare.
        substrings (list of str): Sottostringhe da cercare nei nomi dei file.
    
    Returns:
        list: Una lista di dizionari con i risultati per ciascun file.
    """
    results = []
    
    # Verifica che la directory esista
    if not os.path.isdir(directory_path):
        print(f"La directory '{directory_path}' non esiste.")
        return results
    
    # Itera su tutti i file nella directory
    for filename in os.listdir(directory_path):
        if np.all([s in filename for s in substrings]):  # Controlla la sottostringa nel nome del file
            file_path = os.path.join(directory_path, filename)
            try:
                # Applica la funzione analyze_attempts
                result = analyze_attempts(file_path)
                result["filename"] = filename  # Aggiungi il nome del file ai risultati
                results.append(result)
            except Exception as e:
                print(f"Errore durante l'analisi del file {filename}: {e}")
    
    return results

In [15]:
# results = analyze_all_files_with_substring("./", ["tutteistanze50", "_data"])

# for d in results:
#     print(d["filename"])
#     print("betas: ", d["betas"])
#     print("gammas: ", d["gammas"])
#     print("\n")